<b>

<p>
<center>
<font size="6">
Data Mining Project 3
</font>
</center>
</p>

<p>
<center>
<font size="5">
WebMD Scraping Process
</font>
</center>
</p>

<p>
<center>
<font size="3">
Data Mining, Columbian College of Arts & Sciences, George Washington University
</font>
</center>
</p>

<p>
<center>
<font size="3">
Author: Voratham Tiabrat
</font>
</center>
</p>

</b>

# Part 1 : WebMD Scraping Process 

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np
import time

In [2]:
# Get all common condition from WebMD
web = requests.get('https://www.webmd.com/drugs/2/conditions/index')
condition = []
links = []
data = BeautifulSoup(web.content, 'lxml')
body = data.find('body')
main = body.find('main')
div = main.find("div",attrs={"class": "drugs-common-results"})
for link in div.find_all("a"):
    condition.append(link.text)
    links.append('https://www.webmd.com/'+link.get("href"))

In [3]:
len(condition)

37

In [4]:
len(links)

37

In [5]:
cond= []
review_url = []
drug_url = []
temp = []
i=0

# Collect all information from each condition links
for url in links :
    cond_temp = condition[i]
    try :
        # get cond,drug,indication, Tpye, reviews,review_url,drug_url
        web = requests.get(url)
        data = BeautifulSoup(web.content, 'lxml')
        body = data.find('body')
        main = body.find('main')
        div = main.find("table",attrs={"class": "drugs-treatments-table"})
        for link in div.find_all("a"):
            # collect review url
            if re.match("/drugs/drugreview", link.get('href')):
                review_url.append('https://www.webmd.com/'+link.get('href'))
                cond.append(cond_temp)
                
            # collect description url
            else :
                drug_url.append('https://www.webmd.com/'+link.get('href'))
                
        for link2 in div.find_all("td") :
            temp.append(link2.text)
    
        i+=1
    
    except :
        pass

In [6]:
#temp contain drug name,indication,type and reviews. So we extract to each variables
indication = temp[1::4]
Type = temp[2::4]
reviews = temp[3::4]


#check the length of each list
print(len(cond))
print(len(indication))
print(len(Type))
print(len(reviews))
print(len(review_url))
print(len(drug_url))

7131
7131
7131
7131
7131
7131


In [7]:
# search for drug that have been rated
index = []
for i in range (len(reviews)):
    if reviews[i] != '0 Reviews':
        index.append(i)
len(index)

2730

In [8]:
cond_use= []
indication_use = []
Type_use = []
reviews_use = []
review_url_use = []
drug_url_use = []

for ind in index :
    cond_use.append(cond[ind])
    indication_use.append(indication[ind])
    Type_use.append(Type[ind])
    reviews_use.append(reviews[ind])
    review_url_use.append(review_url[ind])
    drug_url_use.append(drug_url[ind])

In [9]:
#check the length of each list
print(len(cond_use))
print(len(indication_use))
print(len(Type_use))
print(len(reviews_use))
print(len(review_url_use))
print(len(drug_url_use))

2730
2730
2730
2730
2730
2730
2730


In [62]:
#get drug name & price link
drug = []
links2 = []
for url in drug_url_use :
    try :
        web = requests.get(url)
        data = BeautifulSoup(web.content, 'lxml')
        body = data.find('body')
        div = body.find("div",attrs={"class": "drug-names"})
        for name in div.find_all("p"):
            if 'GENERIC' in name.text:
                drug.append(name.text[17:])
                
        div2 = div.find("a",attrs={"class": "drug-lowest-btn"})
        links2.append(div2.get('href'))
    
    except :
        drug.append(None)
        links2.append(None)

In [85]:
print(len(drug))
print(len(links2))

2730
2730


In [106]:
#Get information of drug
infor = []
driver = webdriver.Chrome()

for url in links2 :
    try :
        driver.get(url)
        time.sleep(1)
        page = driver.page_source
        data = BeautifulSoup(page, 'lxml')
        body = data.find('body')
        div = body.find("div",attrs={"id": "main-section"})
        div2 = div.find("p",attrs={"class": "drug-desc"})
        infor.append(div2.text)

    except :
        infor.append(None)

In [107]:
len(infor)

2730

In [108]:
i = 0
for x in infor:
    if x is None:
        i+=1
i

511

In [76]:
#Get effectiveness, ease of use and satisfaction from review_url
effective = []
ease_use = []
satisfaction = []
driver = webdriver.Chrome()

for url in review_url_use :
    try :
        driver.get(url)
        page = driver.page_source
        data = BeautifulSoup(page, 'lxml')
        body = data.find('body')
        div = body.find("div",attrs={"id": "centering_area"})
        eff = div.find("p",attrs={"id":"EffectivenessSummaryValue"})
        easeuse = div.find("p",attrs={"id":"EaseOfUseSummaryValue"})
        satis = div.find("p",attrs={"id":"SideEffectsSummaryValue"})
        effective.append(eff.text.replace('(','').replace(')',''))
        ease_use.append(easeuse.text.replace('(','').replace(')',''))
        satisfaction.append(satis.text.replace('(','').replace(')',''))

    except :
        effective.append(None)
        ease_use.append(None)
        satisfaction.append(None)


In [77]:
print(len(effective))
print(len(ease_use))
print(len(satisfaction))

2730
2730
2730


In [109]:
# Create data frame
join = {'Condition':cond_use,'Drug':drug,'Indication':indication_use,'Type':Type_use,'Reviews':reviews_use,'Effective':effective,'EaseOfUse':ease_use,'Satisfaction':satisfaction,'Information':infor}
df = pd.DataFrame(join)

In [110]:
df

,Condition,Drug,Indication,Type,Reviews,Effective,EaseOfUse,Satisfaction,Information
0,Acute Bacterial Sinusitis,Levofloxacin,On Label,RX,994 Reviews,2.52,3.01,1.84,\n\t\t\t\t\tLevofloxacin is used to treat a va...
1,Acute Bacterial Sinusitis,Levofloxacin,On Label,RX,994 Reviews,2.52,3.01,1.84,\n\t\t\t\t\tLevofloxacin is used to treat a va...
2,Acute Bacterial Sinusitis,Moxifloxacin,On Label,RX,755 Reviews,2.78,3.00,2.08,\n\t\t\t\t\t This is a generic drug. The avera...
3,Acute Bacterial Sinusitis,Azithromycin,On Label,RX,584 Reviews,3.21,4.01,2.57,\n\t\t\t\t\tAzithromycin is an antibiotic (mac...
4,Acute Bacterial Sinusitis,Azithromycin,On Label,RX,584 Reviews,3.21,4.01,2.57,\n\t\t\t\t\tAzithromycin is an antibiotic (mac...
5,Acute Bacterial Sinusitis,Azithromycin,On Label,RX,584 Reviews,3.21,4.01,2.57,\n\t\t\t\t\tAzithromycin is an antibiotic (mac...
6,Acute Bacterial Sinusitis,Azithromycin,On Label,RX,584 Reviews,3.21,4.01,2.57,\n\t\t\t\t\tAzithromycin is an antibiotic (mac...
7,Acute Bacterial Sinusitis,Azithromycin,On Label,RX,584 Reviews,3.21,4.01,2.57,\n\t\t\t\t\tAzithromycin is an antibiotic (mac...
8,Acute Bacterial Sinusitis,Amoxicillin-Pot Clavulanate,On Label,RX,437 Reviews,3.26,3.23,2.42,\n\t\t\t\t\tAmoxicillin/clavulanic acid is a c...
9,Acute Bacterial Sinusitis,Amoxicillin-Pot Clavulanate,On Label,RX,437 Reviews,3.26,3.23,2.42,\n\t\t\t\t\tAmoxicillin/clavulanic acid is a c...


In [114]:
#Clean all None value
df2 = df.dropna(how='any',axis=0)
df2.shape

(2219, 9)

In [116]:
# Export as csv file
df2.to_csv('Drug.csv',index =False)